In [15]:
#Function to return top 10 movies based on a given user's watch history using item based collaborative filtering:

def recommend(user_id):
    ratings=pd.read_csv('ratings.csv')
    ratings.drop('timestamp', inplace=True, axis=1)
    movies=pd.read_csv('movies.csv')
    movies.drop('genres',inplace=True,axis=1)
    ratings=pd.merge(movies,ratings)
    movieRatings=ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
    corrMatrix=movieRatings.corr(method='pearson', min_periods=100)
    myRatings=movieRatings.loc[user_id].dropna()
    simCandidates=pd.Series(dtype="float64")
    same=[]
    for i in range(len(myRatings.index)):
        sims=corrMatrix[myRatings.index[i]].dropna()
        sims=sims.map(lambda x:x*myRatings[i])
        simCandidates=simCandidates.append(sims)
    simCandidates.sort_values(inplace=True, ascending=False)
    simCandidates=simCandidates.groupby(simCandidates.index).sum()
    simCandidates.sort_values(inplace=True, ascending=False)
    print()
    
    same=[]
    for movie in myRatings.index:
        if(movie in simCandidates.index):
            same.append(movie)
            
    ddf=pd.DataFrame(simCandidates)
    filteredCandidates=ddf
    for movie in same:
        filteredCandidates=filteredCandidates.drop(movie)
    filteredCandidates.columns=['Rating']
    print('Top Movies rated by user are :')
    print(myRatings.sort_values(ascending=False).head(10))
    print()
    print('Lowest rated movies by user are :')
    print(myRatings.sort_values(ascending=True).head(10))
    print()
    print("-----------------------------Recommended Movies--------------------------")
    print(filteredCandidates.head(10))

In [16]:
recommend(7)


Top Movies rated by user are :
title
Contact (1997)                                          5.0
Back to the Future (1985)                               5.0
Psycho (1960)                                           5.0
Planet of the Apes (1968)                               5.0
Seven Samurai (Shichinin no samurai) (1954)             5.0
Silence of the Lambs, The (1991)                        5.0
Naked Gun 2 1/2: The Smell of Fear, The (1991)          5.0
Forrest Gump (1994)                                     5.0
Star Wars: Episode IV - A New Hope (1977)               5.0
Spirited Away (Sen to Chihiro no kamikakushi) (2001)    5.0
Name: 7, dtype: float64

Lowest rated movies by user are :
title
Lady in the Water (2006)                               0.5
Pearl Harbor (2001)                                    0.5
Star Wars: Episode II - Attack of the Clones (2002)    0.5
As Good as It Gets (1997)                              0.5
Hulk (2003)                                            0.5
Ho

<ipython-input-15-d57fb6cdaeca>:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  simCandidates=pd.Series()


In [2]:
import pandas as pd
import numpy as np
import sklearn
import timeit


In [3]:
ratings=pd.read_csv('ratings.csv')
ratings.drop('timestamp', inplace=True, axis=1)
movies=pd.read_csv('movies.csv')
movies.drop('genres',inplace=True,axis=1)
ratings=pd.merge(movies,ratings)
ratings.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [7]:
#number of movies
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
dtypes: int64(1), object(1)
memory usage: 152.3+ KB


In [13]:
#number of users
r=ratings['userId'].unique()
len(r)

610

In [4]:
#User-Item matrix (sparse matrix)
movieRatings=ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
movieRatings.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Preparing Correlation Matrix
corrMatrix=movieRatings.corr(method='pearson', min_periods=100)
corrMatrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Movies rated by a particular user :
user_id = 10
myRatings=movieRatings.loc[user_id].dropna()
myRatings

title
27 Dresses (2008)                  3.0
300 (2007)                         3.0
50 First Dates (2004)              3.5
About Time (2013)                  4.0
About a Boy (2002)                 3.5
                                  ... 
What a Girl Wants (2003)           3.0
When Harry Met Sally... (1989)     3.0
White House Down (2013)            4.0
Wolf of Wall Street, The (2013)    1.0
Yes Man (2008)                     0.5
Name: 10, Length: 140, dtype: float64

In [40]:
# Finding similar movies for each movie seen/rated by the user :

simCandidates=pd.Series(dtype="float64")
same=[]
for i in range(len(myRatings.index)):
    
    print("movies similar to ",myRatings.index[i], ":")
    sims=corrMatrix[myRatings.index[i]].dropna()
    # Scaling up the similar movies according to rating giving to the base movie.
    # eg here, user gave the movie "27 Dresses" a 3/5. so all similar movies will be 
    # multiplied by 3 to scale the result and bring out better results on the top
    sims=sims.map(lambda x:x*myRatings[i])
    print(sims)
    simCandidates=simCandidates.append(sims)
    print()

movies similar to  27 Dresses (2008) :
Series([], Name: 27 Dresses (2008), dtype: object)

movies similar to  300 (2007) :
Series([], Name: 300 (2007), dtype: object)

movies similar to  50 First Dates (2004) :
Series([], Name: 50 First Dates (2004), dtype: object)

movies similar to  About Time (2013) :
Series([], Name: About Time (2013), dtype: object)

movies similar to  About a Boy (2002) :
Series([], Name: About a Boy (2002), dtype: object)

movies similar to  Aladdin (1992) :
title
Ace Ventura: Pet Detective (1994)    1.208773
Aladdin (1992)                       4.000000
Apollo 13 (1995)                     1.284887
Batman (1989)                        1.096611
Beauty and the Beast (1991)          2.328745
Braveheart (1995)                    0.679009
Forrest Gump (1994)                  1.857070
Fugitive, The (1993)                 1.124994
Jurassic Park (1993)                 1.638094
Lion King, The (1994)                2.366642
Mask, The (1994)                     1.557661
P

In [42]:
simCandidates.sort_values(inplace=True, ascending=False)
simCandidates=simCandidates.groupby(simCandidates.index).sum()
simCandidates.sort_values(inplace=True, ascending=False)
simCandidates

Lord of the Rings: The Return of the King, The (2003)        14.795471
Lord of the Rings: The Two Towers, The (2002)                14.495176
Lord of the Rings: The Fellowship of the Ring, The (2001)    13.677510
Forrest Gump (1994)                                          12.046372
Matrix, The (1999)                                           11.103098
                                                               ...    
Minority Report (2002)                                        0.126744
Terminator, The (1984)                                        0.125558
X-Men (2000)                                                  0.095920
Blade Runner (1982)                                           0.079321
Fifth Element, The (1997)                                     0.064005
Length: 88, dtype: float64

In [43]:
# Removing movies already seen by user 

same=[]
for movie in myRatings.index:
    if(movie in simCandidates.index):
        same.append(movie)

ddf=pd.DataFrame(simCandidates)
filteredCandidates=ddf
for movie in same:
    filteredCandidates=filteredCandidates.drop(movie)


In [45]:
filteredCandidates.columns=['Rating']
print('Top Movies rated by user are :')
print()
print(myRatings.sort_values(ascending=False).head(10))
print()
print('Lowest rated movies by user are :')
print()
print(myRatings.sort_values(ascending=True).head(10))
print()
print("-----------------------------Recommended Movies--------------------------")
print(filteredCandidates.head(10))

Top Movies rated by user are :

title
First Daughter (2004)            5.0
Skyfall (2012)                   5.0
Spectre (2015)                   5.0
Dark Knight Rises, The (2012)    5.0
King's Speech, The (2010)        5.0
Despicable Me (2010)             5.0
Batman Begins (2005)             5.0
Intouchables (2011)              5.0
The Intern (2015)                5.0
Notebook, The (2004)             5.0
Name: 10, dtype: float64

Lowest rated movies by user are :

title
Yes Man (2008)                               0.5
Interstellar (2014)                          0.5
Julie & Julia (2009)                         0.5
Grand Budapest Hotel, The (2014)             0.5
Mary Poppins (1964)                          0.5
Fight Club (1999)                            0.5
Matrix, The (1999)                           0.5
Sixth Sense, The (1999)                      0.5
Rust and Bone (De rouille et d'os) (2012)    0.5
Kingsman: The Secret Service (2015)          1.0
Name: 10, dtype: float64

---------

In [46]:
# Testing execution time :

code_to_test = """

import pandas as pd
import numpy as np
import sklearn
import timeit

user_id=10
ratings=pd.read_csv('ratings.csv')
ratings.drop('timestamp', inplace=True, axis=1)
movies=pd.read_csv('movies.csv')
movies.drop('genres',inplace=True,axis=1)
ratings=pd.merge(movies,ratings)
movieRatings=ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
corrMatrix=movieRatings.corr(method='pearson', min_periods=100)
myRatings=movieRatings.loc[user_id].dropna()
simCandidates=pd.Series()
same=[]
for i in range(len(myRatings.index)):
    sims=corrMatrix[myRatings.index[i]].dropna()
    sims=sims.map(lambda x:x*myRatings[i])
    simCandidates=simCandidates.append(sims)
simCandidates.sort_values(inplace=True, ascending=False)
simCandidates=simCandidates.groupby(simCandidates.index).sum()
simCandidates.sort_values(inplace=True, ascending=False)

same=[]
for movie in myRatings.index:
    if(movie in simCandidates.index):
        #print(movie)
        same.append(movie)

ddf=pd.DataFrame(simCandidates)
filteredCandidates=ddf
for movie in same:
    filteredCandidates=filteredCandidates.drop(movie)
filteredCandidates.columns=['Rating']
print('Movies rated by user are :')
print(myRatings.head(10))
print()
print("-----------------------------Recommended Movies--------------------------")
print(filteredCandidates.head(10))


"""

In [47]:
elapsed_time = timeit.timeit(code_to_test, number=10)/10
print(elapsed_time)

Movies rated by user are :
title
27 Dresses (2008)                 3.0
300 (2007)                        3.0
50 First Dates (2004)             3.5
About Time (2013)                 4.0
About a Boy (2002)                3.5
Aladdin (1992)                    4.0
Amazing Spider-Man, The (2012)    3.0
American Beauty (1999)            1.0
As Good as It Gets (1997)         3.5
Avatar (2009)                     2.5
Name: 10, dtype: float64

-----------------------------Recommended Movies--------------------------
                                                      Rating
Star Wars: Episode IV - A New Hope (1977)           7.496018
Shawshank Redemption, The (1994)                    7.203107
Raiders of the Lost Ark (Indiana Jones and the ...  6.686895
Silence of the Lambs, The (1991)                    6.642676
Star Wars: Episode V - The Empire Strikes Back ...  6.598600
Star Wars: Episode VI - Return of the Jedi (1983)   6.407184
Pirates of the Caribbean: The Curse of the Blac...  6.371572

Movies rated by user are :
title
27 Dresses (2008)                 3.0
300 (2007)                        3.0
50 First Dates (2004)             3.5
About Time (2013)                 4.0
About a Boy (2002)                3.5
Aladdin (1992)                    4.0
Amazing Spider-Man, The (2012)    3.0
American Beauty (1999)            1.0
As Good as It Gets (1997)         3.5
Avatar (2009)                     2.5
Name: 10, dtype: float64

-----------------------------Recommended Movies--------------------------
                                                      Rating
Star Wars: Episode IV - A New Hope (1977)           7.496018
Shawshank Redemption, The (1994)                    7.203107
Raiders of the Lost Ark (Indiana Jones and the ...  6.686895
Silence of the Lambs, The (1991)                    6.642676
Star Wars: Episode V - The Empire Strikes Back ...  6.598600
Star Wars: Episode VI - Return of the Jedi (1983)   6.407184
Pirates of the Caribbean: The Curse of the Blac...  6.371572